# Challenge 3

In this challenge we will work on the `Orders.csv` data set in the previous [Subsetting and Descriptive Stats lab](../../lab-subsetting-and-descriptive-stats/your-code/main.ipynb). In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

# Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders.csv` from the "subsetting" lab folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# enter your code here
orders = pd.read_csv("../../lab-subsetting-and-descriptive-stats/your-code/Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# your code here

custom_sum_amount=orders.groupby('CustomerID')['amount_spent'].sum().sort_values(ascending=False)
custom_sum_amount.head(5)

CustomerID
14646    280206.02
18102    259657.30
17450    194550.79
16446    168472.50
14911    143825.06
Name: amount_spent, dtype: float64

In [4]:
customers_vip=custom_sum_amount[custom_sum_amount>custom_sum_amount.quantile(0.95)]
customers_preferred=custom_sum_amount[(custom_sum_amount>custom_sum_amount.quantile(0.75)) 
                                      & (custom_sum_amount<custom_sum_amount.quantile(0.95))]

customers_vip.head(5)

CustomerID
14646    280206.02
18102    259657.30
17450    194550.79
16446    168472.50
14911    143825.06
Name: amount_spent, dtype: float64

In [5]:
def func_vip(x):
    if x in customers_vip:
        return 'Vip'
    elif x in customers_preferred:
        return 'Preferred'
    else:
        return 'Regular'

In [6]:
orders['Vip/Preferred']=orders['CustomerID'].apply(func_vip)

In [7]:
orders[orders['Vip/Preferred']=='Vip'].head(5)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,Vip/Preferred
26,26,536370,22728,2010,12,3,8,alarm clock bakelike pink,24,2010-12-01 08:45:00,3.75,12583,France,90.0,Vip
27,27,536370,22727,2010,12,3,8,alarm clock bakelike red,24,2010-12-01 08:45:00,3.75,12583,France,90.0,Vip
28,28,536370,22726,2010,12,3,8,alarm clock bakelike green,12,2010-12-01 08:45:00,3.75,12583,France,45.0,Vip
29,29,536370,21724,2010,12,3,8,panda and bunnies sticker sheet,12,2010-12-01 08:45:00,0.85,12583,France,10.2,Vip
30,30,536370,21883,2010,12,3,8,stars gift tape,24,2010-12-01 08:45:00,0.65,12583,France,15.6,Vip


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

# Q2: How to identify which country has the most VIP Customers?

# Q3: How to identify which country has the most VIP+Preferred Customers combined?

Provide your solution for Q2 below:

In [11]:
# your code here
#Q2:
vips=orders[orders['Vip/Preferred']=='Vip'].groupby('Country')['CustomerID'].count().sort_values(ascending=False)
vips.head(5)

Country
United Kingdom    84185
EIRE               7077
France             3290
Germany            3127
Netherlands        2080
Name: CustomerID, dtype: int64

In [12]:
#Q3
vips_pref=orders[(orders['Vip/Preferred']=='Vip')|(orders['Vip/Preferred']=='Preferred')].groupby('Country')['CustomerID'].count().sort_values(ascending=False)
vips_pref.head(5)

Country
United Kingdom    221635
Germany             7349
EIRE                7238
France              6301
Netherlands         2080
Name: CustomerID, dtype: int64